# Test VCC2018

In [180]:
import models
import dataset
import torch    
from collections import OrderedDict

def extract_prefix(prefix, weights):
    result = OrderedDict()
    for key in weights:
        if key.find(prefix) == 0:
            result[key[len(prefix):]] = weights[key]
    return result     


ds = dataset.VCC2018DatasetNoPreporocess(list_path='../MOSNet/data/mos_list.txt', data_path='../MOSNet/data/wav/')
model = models.NemoMOS().cuda()
model.load_state_dict(extract_prefix('model.', torch.load('/home/user01/Downloads/speakernet-verif-tune.ckpt')['state_dict']), strict=False)
_=model.eval()

[NeMo I 2021-05-28 21:35:08 cloud:56] Found existing object /home/user01/.cache/torch/NeMo/NeMo_1.0.0rc1/speakerverification_speakernet/2202d9ccf2dccac4c87faf6b5507eb9d/speakerverification_speakernet.nemo.
[NeMo I 2021-05-28 21:35:08 cloud:62] Re-using file from: /home/user01/.cache/torch/NeMo/NeMo_1.0.0rc1/speakerverification_speakernet/2202d9ccf2dccac4c87faf6b5507eb9d/speakerverification_speakernet.nemo
[NeMo I 2021-05-28 21:35:08 common:654] Instantiating model from pre-trained checkpoint


[NeMo W 2021-05-28 21:35:08 modelPT:133] Please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /ws/manifests/raid/combined/train_manifest.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 8
    augmentor:
      noise:
        manifest_path: /ws/manifests/raid/musan/musan_music_noise_manifest_dur8.json
        prob: 0.2
        min_snr_db: 5
        max_snr_db: 15
    num_workers: 4
    
[NeMo W 2021-05-28 21:35:08 modelPT:140] Please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /ws/manifests/raid/voxceleb/small_manifest.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: false
    time_length: 8
    num_workers: 1
    


[NeMo I 2021-05-28 21:35:08 features:240] PADDING: 16
[NeMo I 2021-05-28 21:35:08 features:256] STFT using torch
[NeMo I 2021-05-28 21:35:08 label_models:88] Training with Angular Softmax Loss
[NeMo I 2021-05-28 21:35:08 modelPT:376] Model EncDecSpeakerLabelModel was successfully restored from /home/user01/.cache/torch/NeMo/NeMo_1.0.0rc1/speakerverification_speakernet/2202d9ccf2dccac4c87faf6b5507eb9d/speakerverification_speakernet.nemo.
[NeMo I 2021-05-28 21:35:08 features:240] PADDING: 16
[NeMo I 2021-05-28 21:35:08 features:256] STFT using torch


In [78]:
import tqdm
import dataset


pred_mos = []
gt_mos   = []
for i in tqdm.trange(ds.getlen('test')):
    item = ds.getitem('test', i)
    x, l = dataset.collate_fn_lenth([item])[0]
    with torch.no_grad():
        res = model((x.cuda(), l.cuda())).mean()
    
    gt_mos.append(item[1].item())
    pred_mos.append(res.item())

100%|██████████| 4000/4000 [00:33<00:00, 119.64it/s]


In [79]:
import numpy as np
import scipy.stats as ss

mse = ((np.array(pred_mos)-np.array(gt_mos))**2).mean()
lcc = np.corrcoef(np.array(gt_mos), np.array(pred_mos))[0,1]
srcc = ss.spearmanr(np.array(gt_mos), np.array(pred_mos))[0]

In [80]:
ds.samples['test'].columns = ['audio', 'true_mos']
df = ds.samples['test'].copy()
df['predict_mos'] = np.array(pred_mos)

In [81]:
import pandas as pd

sys_df = pd.read_csv('../MOSNet/data/vcc2018_system.csv')
df['system_ID'] = df['audio'].str.split('_').str[-1].str.split('.').str[0] + '_' + df['audio'].str.split('_').str[0]
result_mean = df[['system_ID', 'predict_mos']].groupby(['system_ID']).mean()
mer_df = pd.merge(result_mean, sys_df, on='system_ID')                                                                                                                 

sys_true = mer_df['mean']
sys_predicted = mer_df['predict_mos']

In [82]:
sys_mse = ((sys_true-sys_predicted)**2).mean()
sys_lcc = np.corrcoef(sys_true, sys_predicted)[0,1]
sys_srcc = ss.spearmanr(sys_true, sys_predicted)[0]

In [83]:
print('[Utterance] MSE = {:.4f} LCC = {:.4f} SRCC = {:.4f}'.format(mse, lcc, srcc))
print('[System   ] MSE = {:.4f} LCC = {:.4f} SRCC = {:.4f}'.format(sys_mse, sys_lcc, sys_srcc))

[Utterance] MSE = 0.4912 LCC = 0.7162 SRCC = 0.6971
[System   ] MSE = 0.0811 LCC = 0.9723 SRCC = 0.9442


# Test VCC2016

In [84]:
from io import StringIO

eh1 = pd.read_csv(StringIO('\n'.join([s.strip().replace(':MOS', '') for s in open('mos_EH1.txt', 'r').readlines()])), sep='\t')
eh2 = pd.read_csv(StringIO('\n'.join([s.strip().replace(':MOS', '') for s in open('mos_EH2.txt', 'r').readlines()])), sep='\t')

mos_vcc2016 = pd.DataFrame(pd.concat([eh1, eh2]).mean())
mos_vcc2016.columns = ['system_mos']

In [85]:
import glob

vcc2016 = glob.glob('../vcc2016_submissions/*/*.wav')

df = pd.DataFrame({
    'audio': vcc2016,
    'system': [s.split('/')[-2] for s in vcc2016]
})

In [86]:
import librosa
import scipy.signal

def load(path):
    signal,sr = librosa.load(path, sr=16000)
    #spec = np.abs(librosa.stft(signal, n_fft=512, hop_length=256, win_length=512, window=scipy.signal.hamming).T).astype(np.float32) # [time, 257]

    return torch.as_tensor(signal).unsqueeze(0), torch.tensor([len(signal)], dtype=torch.long)#.unsqueeze(1)

    
pred_mos = []

for i in tqdm.trange(df.shape[0]):
    signal, l = load(df.iloc[i]['audio'])
    
    with torch.no_grad():
        res = model((signal.cuda(), l.cuda())).mean()
    
    pred_mos.append(res.item())

100%|██████████| 26028/26028 [03:30<00:00, 123.79it/s]


In [87]:
df['pred_mos'] = np.array(pred_mos)

In [88]:
gt_mos = np.array(mos_vcc2016.sort_index()['system_mos'])
pred_mos = np.array(df.groupby('system').mean()['pred_mos'])

In [89]:
def score(gt, pred):
    mse = ((gt-pred)**2).mean()
    lcc = np.corrcoef(gt, pred)[0,1]
    srcc = ss.spearmanr(gt, pred)[0]
    print('MSE = {:.4f} LCC = {:.4f} SRCC = {:.4f}'.format(mse, lcc, srcc))
    
score(gt_mos, pred_mos)

MSE = 0.4023 LCC = 0.9126 SRCC = 0.8917


# Something

In [ ]:
import os
import glob

def load(path):
    signal,sr = librosa.load(path, sr=16000)
    #spec = np.abs(librosa.stft(signal, n_fft=512, hop_length=256, win_length=512, window=scipy.signal.hamming).T).astype(np.float32) # [time, 257]
    #signal = signal[10000:60000]
    return torch.as_tensor(signal).unsqueeze(0), torch.tensor([len(signal)], dtype=torch.long)#.unsqueeze(1)


In [153]:

model_scores = {}

for model_name in ['waveglow', 'hifigan_v1', 'hifigan_v2', 'hifigan_v3', 'melgan', 'val_dataset']:
    print(model_name, '...')
    model_scores[model_name] = []
    for f in os.listdir('to_score/' + model_name):
        signal, l = load(os.path.join('to_score', model_name, f))
        with torch.no_grad():
            res = model((signal.cuda(), l.cuda())).mean()
            model_scores[model_name].append(res)
            
for model_name in model_scores:
    print(model_name, torch.tensor(model_scores[model_name]).mean().item())

waveglow ...
hifigan_v1 ...
hifigan_v2 ...
hifigan_v3 ...
melgan ...
val_dataset ...
waveglow 3.3042843341827393
hifigan_v1 3.5140740871429443
hifigan_v2 3.4486920833587646
hifigan_v3 3.422090530395508
melgan 3.4947543144226074
val_dataset 3.455531120300293
